1.

In [4]:
from collections import deque
import time

# BFS from given source s with performance measurement
def bfs_performance(adj, s):
    q = deque()
    visited = [False] * len(adj)

    visited[s] = True
    q.append(s)
    
    max_queue_size = 0  # Track maximum storage used by the queue
    start_time = time.time()  # Start time

    while q:
        max_queue_size = max(max_queue_size, len(q))
        curr = q.popleft()

        for x in adj[curr]:
            if not visited[x]:
                visited[x] = True
                q.append(x)

    end_time = time.time()  # End time
    execution_time = end_time - start_time

    return max_queue_size, execution_time

# Function to add an edge to the graph
def add_edge(adj, u, v):
    adj[u].append(v)
    adj[v].append(u)

# Generate a tree-like graph with given levels and branching factor
def generate_tree_graph(levels, branching_factor):
    adj = []
    node = 0

    for level in range(levels):
        level_nodes = branching_factor ** level
        for i in range(level_nodes):
            while len(adj) <= node:
                adj.append([])  # Ensure adjacency list is large enough
            parent = node
            for _ in range(branching_factor):
                child = len(adj)
                adj.append([])  # Expand adjacency list for the child
                add_edge(adj, parent, child)
                node += 1
    return adj

# Measure performance by varying levels and nodes
if __name__ == "__main__":
    levels_to_test = [8, 10]  # Number of levels in the tree
    branching_factors = [2, 3, 4]  # Branching factor for each node

    for levels in levels_to_test:
        for branching_factor in branching_factors:
            print(f"Testing with Levels: {levels}, Branching Factor: {branching_factor}")
            adj = generate_tree_graph(levels, branching_factor)

            max_queue_size, execution_time = bfs_performance(adj, 0)

            print(f"Max Queue Size (Storage): {max_queue_size}")
            print(f"Execution Time: {execution_time:.6f} seconds")
            print("-" * 50)


Testing with Levels: 8, Branching Factor: 2
Max Queue Size (Storage): 2
Execution Time: 0.000997 seconds
--------------------------------------------------
Testing with Levels: 8, Branching Factor: 3
Max Queue Size (Storage): 3
Execution Time: 0.049969 seconds
--------------------------------------------------
Testing with Levels: 8, Branching Factor: 4
Max Queue Size (Storage): 4
Execution Time: 0.085945 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 2
Max Queue Size (Storage): 2
Execution Time: 0.001996 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 3
Max Queue Size (Storage): 3
Execution Time: 0.083950 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 4
Max Queue Size (Storage): 4
Execution Time: 1.287205 seconds
--------------------------------------------------


In [6]:
import time

# DFS from a given source with performance measurement
def dfs_performance(adj, s):
    stack = []  # Explicit stack for DFS
    visited = [False] * len(adj)

    stack.append(s)
    visited[s] = True
    
    max_stack_size = 0  # Track maximum storage used by the stack
    start_time = time.time()  # Start time

    while stack:
        max_stack_size = max(max_stack_size, len(stack))
        curr = stack.pop()

        for x in adj[curr]:
            if not visited[x]:
                visited[x] = True
                stack.append(x)

    end_time = time.time()  # End time
    execution_time = end_time - start_time

    return max_stack_size, execution_time

# Function to add an edge to the graph
def add_edge(adj, u, v):
    adj[u].append(v)
    adj[v].append(u)

# Generate a tree-like graph with given levels and branching factor
def generate_tree_graph(levels, branching_factor):
    adj = []
    node = 0

    for level in range(levels):
        level_nodes = branching_factor ** level
        for i in range(level_nodes):
            while len(adj) <= node:
                adj.append([])  # Ensure adjacency list is large enough
            parent = node
            for _ in range(branching_factor):
                child = len(adj)
                adj.append([])  # Expand adjacency list for the child
                add_edge(adj, parent, child)
                node += 1
    return adj

# Measure performance by varying levels and nodes
if __name__ == "__main__":
    levels_to_test = [8, 10]  # Number of levels in the tree
    branching_factors = [2, 3, 4]  # Branching factor for each node

    for levels in levels_to_test:
        for branching_factor in branching_factors:
            print(f"Testing with Levels: {levels}, Branching Factor: {branching_factor}")
            adj = generate_tree_graph(levels, branching_factor)

            max_stack_size, execution_time = dfs_performance(adj, 0)

            print(f"Max Stack Size (Storage): {max_stack_size}")
            print(f"Execution Time: {execution_time:.6f} seconds")
            print("-" * 50)


Testing with Levels: 8, Branching Factor: 2
Max Stack Size (Storage): 256
Execution Time: 0.000999 seconds
--------------------------------------------------
Testing with Levels: 8, Branching Factor: 3
Max Stack Size (Storage): 6561
Execution Time: 0.031980 seconds
--------------------------------------------------
Testing with Levels: 8, Branching Factor: 4
Max Stack Size (Storage): 65536
Execution Time: 0.136912 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 2
Max Stack Size (Storage): 1024
Execution Time: 0.001995 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 3
Max Stack Size (Storage): 59049
Execution Time: 0.087945 seconds
--------------------------------------------------
Testing with Levels: 10, Branching Factor: 4
Max Stack Size (Storage): 1048576
Execution Time: 1.377155 seconds
--------------------------------------------------


## 3. Best First

In [14]:
import heapq

# Define the 8-puzzle class
class Puzzle8:
    def __init__(self, initial_state):
        self.goal_state = [1,2,3,8,0,4,7,6,5]
        self.initial_state = initial_state

    # Define the heuristic functions
    def local_heuristic(self, state):
        # local heuristic: Misplaced tiles
        count = 0
        for i in range(9):
            if state[i] != 0 and state[i] != self.goal_state[i]:
                count += 1
        return count

    def global_heuristic(self, state):
        # Global heuristic: Sum of Manhattan distances
        distance = 0
        for i in range(9):
            if state[i] != 0:
                goal_idx = self.goal_state.index(state[i])
                x1, y1 = i // 3, i % 3
                x2, y2 = goal_idx // 3, goal_idx % 3
                distance += abs(x1 - x2) + abs(y1 - y2)
        return distance

    def is_goal_state(self, state):
        return state == self.goal_state

    def generate_children(self, state):
        children = []
        zero_idx = state.index(0)
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up
        for move in moves:
            new_x = zero_idx // 3 + move[0]
            new_y = zero_idx % 3 + move[1]
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_state = state[:]
                new_zero_idx = new_x * 3 + new_y
                new_state[zero_idx], new_state[new_zero_idx] = (
                    new_state[new_zero_idx],
                    new_state[zero_idx],
                )
                children.append(new_state)
        return children

    def best_first_search(self, heuristic):
        open_list = [(heuristic(self.initial_state), self.initial_state, [])]
        closed_set = set()

        while open_list:
            _, current_state, path = heapq.heappop(open_list)

            if self.is_goal_state(current_state):
                return path + [current_state]

            if tuple(current_state) in closed_set:
                continue

            closed_set.add(tuple(current_state))

            for child_state in self.generate_children(current_state):
                if tuple(child_state) not in closed_set:
                    heapq.heappush(
                        open_list,
                        (heuristic(child_state), child_state, path + [current_state]),
                    )

        return None

# Example usage
if __name__ == "__main__":
    initial_state = [0,8,3,2,6,4,1,7,5]
    puzzle = Puzzle8(initial_state)

    # Using global heuristic (h1)
    moves_h1 = puzzle.best_first_search(puzzle.global_heuristic)
    print("Solution using global heuristic (h1):")
    for step in moves_h1:
        print(step)
    print()
    
    # Using local heuristic (h2)
    moves_h2 = puzzle.best_first_search(puzzle.local_heuristic)
    print("Solution using local heuristic (h2):")
    for step in moves_h2:
        print(step)
        # print()



Solution using global heuristic (h1):
[0, 8, 3, 2, 6, 4, 1, 7, 5]
[2, 8, 3, 0, 6, 4, 1, 7, 5]
[2, 8, 3, 1, 6, 4, 0, 7, 5]
[2, 8, 3, 1, 6, 4, 7, 0, 5]
[2, 8, 3, 1, 0, 4, 7, 6, 5]
[2, 0, 3, 1, 8, 4, 7, 6, 5]
[0, 2, 3, 1, 8, 4, 7, 6, 5]
[1, 2, 3, 0, 8, 4, 7, 6, 5]
[1, 2, 3, 8, 0, 4, 7, 6, 5]

Solution using local heuristic (h2):
[0, 8, 3, 2, 6, 4, 1, 7, 5]
[2, 8, 3, 0, 6, 4, 1, 7, 5]
[2, 8, 3, 1, 6, 4, 0, 7, 5]
[2, 8, 3, 1, 6, 4, 7, 0, 5]
[2, 8, 3, 1, 0, 4, 7, 6, 5]
[2, 0, 3, 1, 8, 4, 7, 6, 5]
[0, 2, 3, 1, 8, 4, 7, 6, 5]
[1, 2, 3, 0, 8, 4, 7, 6, 5]
[1, 2, 3, 8, 0, 4, 7, 6, 5]


## 4. Water Jug

In [ ]:
4)To implement A* algorithm for Water-Jug Problem with local & global heuristic functions.	


import heapq

class WaterJug:
    def __init__(self, jug1_capacity, jug2_capacity, goal):
        self.jug1_capacity = jug1_capacity
        self.jug2_capacity = jug2_capacity
        self.goal = goal

    def heuristic(self, state):
        """
        Heuristic function combining local and global heuristics.
        """
        return self.local_heuristic(state) + self.global_heuristic(state)

    def local_heuristic(self, state):
        """
        Local heuristic function.
        Example: Difference from goal.
        """
        return abs(state[0] - self.goal) + abs(state[1] - self.goal)

    def global_heuristic(self, state):
        """
        Global heuristic function.
        Example: Total amount of water in both jugs.
        """
        return abs(state[0] + state[1] - self.goal)

    def is_goal(self, state):
        return state[0] == self.goal or state[1] == self.goal

    def get_neighbors(self, state):
        """
        Generate neighboring states.
        """
        jug1, jug2 = state
        neighbors = []
        
        # Fill Jug1
        if jug1 < self.jug1_capacity:
            neighbors.append((self.jug1_capacity, jug2))
        
        # Fill Jug2
        if jug2 < self.jug2_capacity:
            neighbors.append((jug1, self.jug2_capacity))
        
        # Empty Jug1
        if jug1 > 0:
            neighbors.append((0, jug2))
        
        # Empty Jug2
        if jug2 > 0:
            neighbors.append((jug1, 0))
        
        # Pour Jug1 -> Jug2
        if jug1 > 0 and jug2 < self.jug2_capacity:
            pour_amount = min(jug1, self.jug2_capacity - jug2)
            neighbors.append((jug1 - pour_amount, jug2 + pour_amount))
        
        # Pour Jug2 -> Jug1
        if jug2 > 0 and jug1 < self.jug1_capacity:
            pour_amount = min(jug2, self.jug1_capacity - jug1)
            neighbors.append((jug1 + pour_amount, jug2 - pour_amount))

        return neighbors

    def a_star(self, initial_state):
        open_list = []
        heapq.heappush(open_list, (self.heuristic(initial_state), initial_state))
        came_from = {}
        g_score = {initial_state: 0}
        f_score = {initial_state: self.heuristic(initial_state)}

        while open_list:
            current = heapq.heappop(open_list)[1]

            if self.is_goal(current):
                return self.reconstruct_path(came_from, current)

            for neighbor in self.get_neighbors(current):
                tentative_g_score = g_score[current] + 1  # Assuming uniform cost for simplicity

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

        return None

    def reconstruct_path(self, came_from, current):
        path = []
        while current in came_from:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path

# Example usage
jug1_capacity = 4
jug2_capacity = 3
goal = 2
initial_state = (0, 0)

water_jug_problem = WaterJug(jug1_capacity, jug2_capacity, goal)
path = water_jug_problem.a_star(initial_state)
print("Path to goal:", path)


In [ ]:
import heapq

# Function to perform A* for the Water Jug Problem
def a_star_water_jug(capacity1, capacity2, target, heuristic):
    open_list = []
    parent = {}  # To reconstruct the path
    cost = {}  # To store the cost to reach each state

    # Initialize the starting state
    start = (0, 0)
    heapq.heappush(open_list, (0, start))
    parent[start] = None
    cost[start] = 0

    while open_list:
        _, current_state = heapq.heappop(open_list)
        jug1, jug2 = current_state

        # If the target is achieved
        if jug1 == target or jug2 == target:
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent[current_state]
            return path[::-1]  # Reverse the path

        # Generate possible moves
        transitions = [
            (capacity1, jug2),  # Fill Jug 1
            (jug1, capacity2),  # Fill Jug 2
            (0, jug2),          # Empty Jug 1
            (jug1, 0),          # Empty Jug 2
            (max(0, jug1 - (capacity2 - jug2)), min(capacity2, jug1 + jug2)),  # Pour Jug 1 -> Jug 2
            (min(capacity1, jug1 + jug2), max(0, jug2 - (capacity1 - jug1)))   # Pour Jug 2 -> Jug 1
        ]

        for next_state in transitions:
            if next_state not in cost or cost[current_state] + 1 < cost[next_state]:
                cost[next_state] = cost[current_state] + 1
                priority = cost[next_state] + heuristic(next_state, target)
                heapq.heappush(open_list, (priority, next_state))
                parent[next_state] = current_state

    return None  # Return None if no solution is found

# Heuristic functions
def global_heuristic(state, target):
    jug1, jug2 = state
    return abs(target - jug1) + abs(target - jug2)

def local_heuristic(state, target):
    jug1, jug2 = state
    return min(abs(target - jug1), abs(target - jug2))

# Example Usage
capacity1 = 3  # Capacity of the 3-liter jug
capacity2 = 4  # Capacity of the 5-liter jug
target = 2     # Target amount to measure

print("Using Global Heuristic:")
solution_global = a_star_water_jug(capacity1, capacity2, target, global_heuristic)
if solution_global:
    print("Solution steps:", solution_global)
else:
    print("No solution found.")

print("\nUsing Local Heuristic:")
solution_local = a_star_water_jug(capacity1, capacity2, target, local_heuristic)
if solution_local:
    print("Solution steps:", solution_local)
else:
    print("No solution found.")


Using Global Heuristic:
Solution steps: [(0, 0), (3, 0), (0, 3), (3, 3), (2, 4)]

Using Local Heuristic:
Solution steps: [(0, 0), (3, 0), (0, 3), (3, 3), (2, 4)]


In [3]:
class Node:
    def __init__(self, data, level, fval):
        """ Initialize the node with the data, level of the node, and the calculated f-value."""
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self):
        """ Generate child nodes by moving the blank space in all possible directions."""
        x, y = self.find(self.data, '0')
        moves = [[x, y - 1], [x, y + 1], [x - 1, y], [x + 1, y]]  # Left, Right, Up, Down
        children = []
        for move in moves:
            child = self.shuffle(self.data, x, y, move[0], move[1])
            if child is not None:
                child_node = Node(child, self.level + 1, 0)
                children.append(child_node)
        return children

    def shuffle(self, puz, x1, y1, x2, y2):
        """ Swap the blank space in the specified direction."""
        if 0 <= x2 < len(self.data) and 0 <= y2 < len(self.data):
            temp_puz = self.copy(puz)
            temp_puz[x2][y2], temp_puz[x1][y1] = temp_puz[x1][y1], temp_puz[x2][y2]
            return temp_puz
        else:
            return None

    def copy(self, root):
        """ Create a copy of the matrix."""
        return [row[:] for row in root]

    def find(self, puz, x):
        """ Find the position of the blank space."""
        for i in range(len(self.data)):
            for j in range(len(self.data)):
                if puz[i][j] == x:
                    return i, j


class Puzzle:
    def __init__(self, size):
        """ Initialize the puzzle size, open and closed lists."""
        self.n = size
        self.open = []
        self.closed = []
        self.visited_states = []

    def misplaced_tiles(self, start, goal):
        """ Heuristic: Count the number of misplaced tiles."""
        return sum(start[i][j] != goal[i][j] and start[i][j] != '0' for i in range(self.n) for j in range(self.n))

    def manhattan_distance(self, start, goal):
        """ Heuristic: Calculate the Manhattan distance for each tile."""
        distance = 0
        goal_positions = {goal[i][j]: (i, j) for i in range(self.n) for j in range(self.n)}
        for i in range(self.n):
            for j in range(self.n):
                if start[i][j] != '0' and start[i][j] in goal_positions:
                    x, y = goal_positions[start[i][j]]
                    distance += abs(x - i) + abs(y - j)
        return distance

    def f(self, start, goal, heuristic):
        """ f(x) = h(x) + g(x) """
        return heuristic(start.data, goal) + start.level

    def process(self, heuristic, heuristic_name):
        """ Solve the puzzle using the specified heuristic."""
        # Static input for initial and goal states
        start = [
            ['1', '2', '3'],
            ['8', '0', '4'],
            ['7', '6', '5']
        ]
        goal = [
            ['1', '2', '3'],
            ['8', '4', '5'],
            ['7', '0', '6']
        ]

        start = Node(start, 0, 0)
        start.fval = self.f(start, goal, heuristic)
        self.open.append(start)

        print(f"\nSolution Path using {heuristic_name} Heuristic:\n")
        while self.open:
            cur = self.open[0]
            self.visited_states.append(cur.data)
            print("\nCurrent State:")
            for row in cur.data:
                print(' '.join(row))

            if heuristic(cur.data, goal) == 0:
                print("\nGoal state reached!")
                break

            for child in cur.generate_child():
                child.fval = self.f(child, goal, heuristic)
                self.open.append(child)
            self.closed.append(cur)
            del self.open[0]
            self.open.sort(key=lambda x: x.fval)

        print("\nAll Visited States:")
        for state in self.visited_states:
            for row in state:
                print(' '.join(row))
            print("---")


# Create the puzzle object
puz = Puzzle(3)

# Solve using Misplaced Tiles heuristic
puz.process(puz.misplaced_tiles, "Misplaced Tiles")

# Solve using Manhattan Distance heuristic
puz.process(puz.manhattan_distance, "Manhattan Distance")



Solution Path using Misplaced Tiles Heuristic:


Current State:
1 2 3
8 0 4
7 6 5

Current State:
1 2 3
8 4 0
7 6 5

Current State:
1 2 3
8 4 5
7 6 0

Current State:
1 2 3
8 4 5
7 0 6

Goal state reached!

All Visited States:
1 2 3
8 0 4
7 6 5
---
1 2 3
8 4 0
7 6 5
---
1 2 3
8 4 5
7 6 0
---
1 2 3
8 4 5
7 0 6
---

Solution Path using Manhattan Distance Heuristic:


Current State:
1 2 3
8 4 5
7 0 6

Goal state reached!

All Visited States:
1 2 3
8 0 4
7 6 5
---
1 2 3
8 4 0
7 6 5
---
1 2 3
8 4 5
7 6 0
---
1 2 3
8 4 5
7 0 6
---
1 2 3
8 4 5
7 0 6
---


In [4]:
import copy
import time


class State:
    def __init__(self, first_stack, second_stack, total_num, moves=None):
        if moves is None:
            moves = []
        self.first_stack = first_stack
        self.second_stack = second_stack
        self.total_num = total_num
        self.moves = moves

    def __eq__(self, other):
        return (
            self.first_stack == other.first_stack
            and self.second_stack == other.second_stack
            and self.total_num == other.total_num
            and self.moves == other.moves
        )

    def goal_state_move(self, visited_states, heuristic_func):
        while self.difference() != 0:
            visited_states.append((copy.deepcopy(self.first_stack), heuristic_func(self)))
            self = self.select_move(heuristic_func)
        visited_states.append((copy.deepcopy(self.first_stack), heuristic_func(self)))  # Final state
        return self.moves

    def select_move(self, heuristic_func):
        best_move = None
        best_score = float("inf")

        # Generate valid moves and select the one with the best heuristic score
        for index, stack in enumerate(self.first_stack):
            for index2, stack2 in enumerate(self.first_stack):
                if index != index2:
                    curr_table, move = self.valid_state_move(self.first_stack, index, index2)
                    new_state = State(curr_table, self.second_stack, self.total_num, copy.copy(self.moves))
                    new_state.moves.append(move)
                    score = heuristic_func(new_state)
                    if score < best_score:
                        best_move = new_state
                        best_score = score

        # Move to temporary table if no improvement
        for index, stack in enumerate(self.first_stack):
            if len(stack) > 1:
                curr_table, move = self.valid_state_move(self.first_stack, index, -1)
                new_state = State(curr_table, self.second_stack, self.total_num, copy.copy(self.moves))
                new_state.moves.append(move)
                score = heuristic_func(new_state)
                if score < best_score:
                    best_move = new_state
                    best_score = score

        return best_move

    def valid_state_move(self, table, start_index, end_index):
        temp_table = copy.deepcopy(table)
        left = temp_table[start_index]
        top_block = left.pop()
        if end_index < 0:  # Move to temporary table
            temp_table.append([top_block])
            move = (top_block, "Table")
        else:  # Move to another stack
            right = temp_table[end_index]
            move = (top_block, right[-1] if right else "Empty Stack")
            right.append(top_block)
        if len(left) == 0:
            temp_table.remove(left)
        return temp_table, move

    def difference(self):
        same_num = 0
        for left in self.first_stack:
            for right in self.second_stack:
                index = 0
                while index < len(left) and index < len(right):
                    if left[index] == right[index]:
                        same_num += 1
                        index += 1
                    else:
                        break
        diff = self.total_num - same_num
        return diff


class BlockWorldAgent:
    def __init__(self):
        self.visited_states = []

    def solve(self, initial_arrangement, goal_arrangement, heuristic_func):
        start_time = time.time()
        total_num = sum(len(stack) for stack in initial_arrangement)
        state = State(initial_arrangement, goal_arrangement, total_num)
        solution = state.goal_state_move(self.visited_states, heuristic_func)

        end_time = time.time()
        runtime = str((end_time - start_time) * 1000)
        print("\nSolution Moves:", solution)
        print("Running Time:", runtime, "ms")

        print("\nVisited States:")
        for step, (state, score) in enumerate(self.visited_states):
            print(f"Step {step + 1}: {state} | Heuristic Value: {score}")

        return solution


def local_heuristic(state):
    """Local heuristic evaluates the difference from the goal state."""
    return state.difference()


def global_heuristic(state):
    """Global heuristic evaluates the difference plus the path cost."""
    return state.difference() + len(state.moves)


def test():
    test_agent = BlockWorldAgent()
    initial_arrangement = [["A", "B", "C"], ["D", "E"]]
    goal_arrangement = [["A", "C"], ["D", "E", "B"]]

    print("\nUsing Local Heuristic:")
    print("Initial Arrangement:", initial_arrangement)
    print("Goal Arrangement:", goal_arrangement)
    test_agent.solve(initial_arrangement, goal_arrangement, local_heuristic)

    print("\nUsing Global Heuristic:")
    print("Initial Arrangement:", initial_arrangement)
    print("Goal Arrangement:", goal_arrangement)
    test_agent.visited_states = []  # Reset visited states
    test_agent.solve(initial_arrangement, goal_arrangement, global_heuristic)


if __name__ == "__main__":
    test()



Using Local Heuristic:
Initial Arrangement: [['A', 'B', 'C'], ['D', 'E']]
Goal Arrangement: [['A', 'C'], ['D', 'E', 'B']]


KeyboardInterrupt: 